In [2]:
!pip install -q langchain langchain-community openai gradio
import os
import re
import gradio as gr
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
import json
import bcrypt # For password hashing

# --- Backend: User Management (Simplified using JSON file) ---\
USERS_FILE = 'users.json'

def load_users():
    """Loads user data from the JSON file."""
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_users(users):
    """Saves user data to the JSON file."""
    with open(USERS_FILE, 'w') as f:
        json.dump(users, f, indent=4)

def hash_password(password):
    """Hashes a password using bcrypt."""
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')

def check_password(password, hashed_password):
    """Checks if a password matches a hashed password."""
    return bcrypt.checkpw(password.encode('utf-8'), hashed_password.encode('utf-8'))

def register_user_backend(username, password):
    """Registers a new user."""
    users = load_users()
    if username in users:
        return False, "Username already exists."
    if len(username) < 3:
        return False, "Username must be at least 3 characters long."
    if len(password) < 6:
        return False, "Password must be at least 6 characters long."

    hashed_pass = hash_password(password)
    users[username] = {'password_hash': hashed_pass}
    save_users(users)
    return True, "Registration successful! You can now log in."

def authenticate_user_backend(username, password):
    """Authenticates an existing user."""
    users = load_users()
    user_data = users.get(username)
    if not user_data:
        return False, "Invalid username or password."

    if check_password(password, user_data['password_hash']):
        return True, "Login successful!"
    return False, "Invalid username or password."

# --- LangChain & Recommendation Backend ---\
# Set OpenAI API key
#INSERT YOUR API KEY
os.environ['OPENAI_API_KEY'] = "YOUR API KEY"

# Initialize LLM
llm = OpenAI(temperature=0.7) # Consider lowering this to 0.5 or 0.6 for more consistent output

# Prompt Template
prompt_template = PromptTemplate(
    input_variables=[
        'age', 'gender', 'weight_kg', 'height_cm', 'veg_or_nonveg', 'disease',
        'region', 'allergics', 'foodtype', 'budget',
        'fitness_level', 'fitness_goals', 'availability'
    ],
    template="""
    Personalized Wellness Recommendation System:
    Based on the following user details, provide tailored recommendations.

    User Profile:
    - Age: {age} years
    - Gender: {gender}
    - Weight: {weight_kg} kg
    - Height: {height_cm} cm
    - Dietary Preference: {veg_or_nonveg}
    - Health Conditions/Impairments: {disease}
    - Allergies: {allergics}
    - Food Preferences/Cuisine: {foodtype}
    - Budget for Gym/Wellness: {budget}
    - Current Fitness Level: {fitness_level}
    - Fitness Goals: {fitness_goals}
    - Available Days for Workouts: {availability} (Note: This input is for context; specific scheduling is beyond this output.)
    - Location (Postcode/Region): {region}

    Please provide the following recommendations, formatted clearly with distinct headings. Aim for specific, actionable suggestions.
    **It is CRITICAL that you use the EXACT headings provided below and ensure all sections are populated with exactly 3 numbered items (1., 2., 3.), even if with general suggestions if specific data is unavailable. Do not include any other text or sections.**

    Gym Recommendations:
    Provide **3** gym names near {region}. Prioritize options that are disability-friendly if {disease} suggests. Otherwise aim for specific proximity based on {region} and what memberships fall within {budget}
    1. [Gym Name 1] - [price of membership, proximity to postcode]
    2. [Gym Name 2] - [price of membership, proximity to postcode]
    3. [Gym Name 3] - [price of membership, proximity to postcode]

    Breakfast Recommendations:
    Provide **3** diverse breakfast meal suggestions considering {veg_or_nonveg}, {allergics}, and {foodtype}. Include macronutrient focus (e.g., "high protein", "balanced", "low carb").
    1. [Breakfast Meal 1] - [brief macronutrient breakdown]
    2. [Breakfast Meal 2] - [brief macronutrient breakdown]
    3. [Breakfast Meal 3] - [brief macronutrient breakdown]

    Dinner Recommendations:
    Provide **3** diverse dinner meal suggestions considering {veg_or_nonveg}, {allergics}, and {foodtype}. Include macronutrient focus.
    1. [Dinner Meal 1] - [brief macronutrient breakdown]
    2. [Dinner Meal 2] - [brief macronutrient breakdown]
    3. [Dinner Meal 3] - [brief macronutrient breakdown]

    Workout Recommendations:
    Provide **3** specific workout routines or types based on {fitness_level}, {fitness_goals}, and {disease}. Include brief explanation on how it achieves {fitness_goals}
    1. [Workout Routine 1] - [brief explanation]
    2. [Workout Routine 2] - [brief explanation]
    3. [Workout Routine 3] - [brief explanation]
    """
)

# LLM Chain
chain = LLMChain(llm=llm, prompt=prompt_template)

# Processing function for recommendations
def generate_recommendations_logic(
    age, gender, weight_val, weight_unit, height_val, height_unit, veg_or_nonveg, disease,
    region, allergics, foodtype, budget, fitness_level, fitness_goals, availability
):
    # Convert weight to kg
    weight_kg = weight_val
    if weight_unit == "Stone":
        weight_kg = weight_val * 6.35029 # 1 stone = 6.35029 kg

    # Convert height to cm
    height_cm = height_val
    if height_unit == "Feet":
        height_cm = height_val * 30.48 # 1 foot = 30.48 cm

    # Cleaned up budget mapping (removed duplicates)
    budget_map = {
        'Under £30/month': 'under 30/month',
        '£30 - £60/month': '30-60/month',
        'Over £60/month': 'over 60/month',
        'Flexible': 'Flexible'
    }
    budget_for_llm = budget_map.get(budget, 'Flexible')

    input_data = {
        "age": age,
        "gender": gender,
        "weight_kg": f"{weight_kg:.2f}", # Format to 2 decimal places
        "height_cm": f"{height_cm:.2f}", # Format to 2 decimal places
        "veg_or_nonveg": veg_or_nonveg,
        "disease": disease if disease else "None",
        "region": region,
        "allergics": allergics if allergics else "None",
        "foodtype": foodtype if foodtype else "Any",
        "budget": budget_for_llm, # Use the mapped budget
        "fitness_level": fitness_level,
        "fitness_goals": ", ".join(fitness_goals) if fitness_goals else "Not specified", # Handle multiselect
        "availability": ", ".join(availability) if availability else "Not specified",
    }

    print("\n--- Input Data to LLM ---")
    print(input_data)
    print("-------------------------\n")

    result = chain.run(input_data)

    print("\n--- Raw LLM Result ---")
    print(result)
    print("----------------------\n")

    # --- MORE ROBUST Regex for Extraction ---
    # Define a helper function for extracting a section
    def extract_section(full_text, start_header, next_header=None):
        pattern = re.escape(start_header) + r'\s*\n(.*?)(?='
        if next_header:
            pattern += re.escape(next_header) + r'|$)'
        else:
            pattern += r'$)' # Match until end of string if no next header

        match = re.search(pattern, full_text, re.DOTALL)
        if match:
            content = match.group(1).strip()
            # Clean each line and filter out empty ones
            cleaned_lines = [line.strip() for line in content.split('\n') if line.strip()]
            return "\n".join(cleaned_lines) if cleaned_lines else "No recommendations found for this section."
        return "No recommendations found for this section."

    # Extract each section sequentially
    gym_output_text = extract_section(result, "Gym Recommendations:", "Breakfast Recommendations:")
    breakfast_output_text = extract_section(result, "Breakfast Recommendations:", "Dinner Recommendations:")
    dinner_output_text = extract_section(result, "Dinner Recommendations:", "Workout Recommendations:")
    workout_output_text = extract_section(result, "Workout Recommendations:") # Workout is the last section, so no next header

    return gym_output_text, breakfast_output_text, dinner_output_text, workout_output_text


# --- Frontend: Gradio Application (remains the same) ---\
with gr.Blocks(title="FitForAll: Login & Recommendations") as demo:
    current_user = gr.State(None)
    login_section = gr.Group(visible=True)
    register_section = gr.Group(visible=False)
    recommendation_app_section = gr.Group(visible=False)

    gr.Markdown("# FitForAll: Personalized Wellness System")
    gr.Markdown("Welcome! Please login or register to get started.")

    with login_section:
        gr.Markdown("##  Login to Your Account")
        login_username_input = gr.Textbox(label="Username", placeholder="Enter your username")
        login_password_input = gr.Textbox(label="Password", type="password", placeholder="Enter your password")
        login_status_output = gr.Markdown("")
        login_button = gr.Button("Login")
        gr.Markdown("---")
        gr.Markdown("Don't have an account?")
        go_to_register_button = gr.Button("Create New Account")

    with register_section:
        gr.Markdown("##  Create New Account")
        register_username_input = gr.Textbox(label="Choose Username", placeholder="Minimum 3 characters")
        register_password_input = gr.Textbox(label="Choose Password", type="password", placeholder="Minimum 6 characters")
        register_confirm_password_input = gr.Textbox(label="Confirm Password", type="password", placeholder="Retype your password")
        register_status_output = gr.Markdown("")
        register_button = gr.Button("Register")
        gr.Markdown("---")
        go_to_login_button = gr.Button("Back to Login")

    with recommendation_app_section:
        gr.Markdown("---")
        gr.Markdown("##  Your Fitness Information")
        with gr.Row():
            age_input = gr.Number(label="Age", info="Your age in years", interactive=True)
            gender_input = gr.Radio(["Male", "Female", "Other"], label="Gender", info="Your biological gender")
        with gr.Row():
            weight_val_input = gr.Number(label="Weight Value", info="Enter your weight")
            weight_unit_input = gr.Radio(["kg", "Stone"], label="Weight Unit", value="kg")
            height_val_input = gr.Number(label="Height Value", info="Enter your height")
            height_unit_input = gr.Radio(["cm", "Feet"], label="Height Unit", value="cm")
        with gr.Row():
            fitness_level_input = gr.Dropdown(
                ["Beginner", "Intermediate", "Advanced", "Sedentary"],
                label="Current Fitness Level",
                info="Select your current activity level",
                multiselect=False
            )
            fitness_goals_input = gr.Dropdown(
                ["Weight Loss", "Muscle Gain", "Endurance Improvement", "General Fitness", "Rehabilitation", "Stress Reduction"],
                label="Fitness Goals",
                info="Select your primary fitness goals",
                multiselect=True
            )
        disease_input = gr.Textbox(label="Disabilities / Health Conditions / Impairments", placeholder="e.g., knee injury, diabetes, wheelchair user", info="Any conditions that might affect your fitness activities or diet.")
        gr.Markdown("---")

        gr.Markdown("##  Your Dietary Information")
        with gr.Row():
            veg_or_nonveg_input = gr.Radio(["Veg", "Non-Veg", "Vegan", "Pescatarian"], label="Dietary Preference", value="Veg")
            allergics_input = gr.Textbox(label="Allergies", placeholder="e.g., Peanuts, Dairy, Gluten", info="List any food allergies.")
        foodtype_input = gr.Textbox(label="Food Preferences (e.g., Asian, Mediterranean)", placeholder="e.g., Spicy, Italian, Indian", info="Any specific cuisines or types of food you prefer.")
        gr.Markdown("---")

        gr.Markdown("##  Logistic and Preference Information")
        with gr.Row():
            region_input = gr.Textbox(label="Postcode / Region", placeholder="e.g., SW1A 0AA, Central London", info="Your postcode or general region for gym recommendations.")
            budget_input = gr.Dropdown(
                ["Under £30/month", "£30 - £60/month", "Over £60/month", "Flexible"],
                label="Budget for Wellness",
                info="Select your preferred budget range for gym and meal considerations.",
                value="Flexible"
            )

        gr.Markdown("###  Your Availability")
        availability_input = gr.CheckboxGroup(
            ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
            label="Days You Are Available",
            info="Select the days of the week you are generally available for workouts.",
            value=["Monday", "Wednesday", "Friday"]
        )
        gr.Markdown("---")

        gr.Markdown("##  Your Personalized Recommendations")
        with gr.Column():
            gym_output = gr.Textbox(label="Gym Recommendations", interactive=False)
            breakfast_output = gr.Textbox(label="Breakfast Recommendations", interactive=False)
            dinner_output = gr.Textbox(label="Dinner Recommendations", interactive=False)
            workout_output = gr.Textbox(label="Workout Recommendations", interactive=False)

        get_recommendations_button = gr.Button("Get My Recommendations")

        gr.Markdown("---")
        gr.Markdown("###  Future Features (Beyond MVP)")
        gr.Markdown("The full **Weekly Schedule** integration with dynamic meal and workout planning based on your preferences and availability, as well as the ability to select preferred recommendations for a tailored schedule, are planned for future iterations.")
        gr.Button("Export/Save Recommendations (Coming Soon)").click(
            fn=lambda: gr.Info("Export feature coming soon!"),
            inputs=[],
            outputs=[]
        )
        gr.Markdown("---")
        app_logout_button = gr.Button("Logout")

    def handle_login(username, password):
        success, message = authenticate_user_backend(username, password)
        if success:
            return (
                username,
                gr.update(value="", visible=False),
                gr.update(value="", visible=False),
                gr.update(value="", visible=False),
                gr.update(visible=False),
                gr.update(visible=False),
                gr.update(visible=True)
            )
        else:
            return (
                None,
                gr.update(),
                gr.update(),
                gr.update(value=message, visible=True),
                gr.update(), gr.update(), gr.update()
            )

    login_button.click(
        fn=handle_login,
        inputs=[login_username_input, login_password_input],
        outputs=[current_user, login_username_input, login_password_input, login_status_output, login_section, register_section, recommendation_app_section]
    )

    def handle_register(username, password, confirm_password):
        if password != confirm_password:
            return (
                gr.update(value="Error: Passwords do not match.", visible=True),
                gr.update(visible=False),
                gr.update(visible=True),
                gr.update(visible=False),
                gr.update()
            )

        success, message = register_user_backend(username, password)
        if success:
            return (
                gr.update(value=message, visible=True),
                gr.update(visible=True),
                gr.update(visible=False),
                gr.update(visible=False),
                gr.update(value="")
            )
        else:
            return (
                gr.update(value=message, visible=True),
                gr.update(visible=False),
                gr.update(visible=True),
                gr.update(visible=False),
                gr.update()
            )

    register_button.click(
        fn=handle_register,
        inputs=[register_username_input, register_password_input, register_confirm_password_input],
        outputs=[register_status_output, login_section, register_section, recommendation_app_section, register_username_input]
    )

    def navigate_to_register():
        return (
            gr.update(visible=False),
            gr.update(visible=True),
            gr.update(visible=False)
        )

    go_to_register_button.click(
        fn=navigate_to_register,
        inputs=[],
        outputs=[login_section, register_section, recommendation_app_section]
    )

    def navigate_to_login():
        return (
            gr.update(visible=True),
            gr.update(visible=False),
            gr.update(visible=False)
        )

    go_to_login_button.click(
        fn=navigate_to_login,
        inputs=[],
        outputs=[login_section, register_section, recommendation_app_section]
    )

    def handle_logout():
        return (
            None,
            gr.update(visible=True),
            gr.update(visible=False),
            gr.update(visible=False)
        )

    app_logout_button.click(
        fn=handle_logout,
        inputs=[],
        outputs=[current_user, login_section, register_section, recommendation_app_section]
    )

    get_recommendations_button.click(
        fn=generate_recommendations_logic,
        inputs=[
            age_input, gender_input, weight_val_input, weight_unit_input,
            height_val_input, height_unit_input, veg_or_nonveg_input, disease_input,
            region_input, allergics_input, foodtype_input, budget_input,
            fitness_level_input, fitness_goals_input, availability_input
        ],
        outputs=[gym_output, breakfast_output, dinner_output, workout_output]
    )

if __name__ == "__main__":
    demo.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'bcrypt'

In [ ]:
demo.close()

Closing server running on port: 7860
